In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sp
from copy import deepcopy
import warnings 
import os
from sklearn.model_selection import KFold
import json
warnings.filterwarnings("ignore") 
import random
random.seed(1234)
np.random.seed(1234)

In [2]:
def obtain_data(task, isbalance, balance):
    
    pwd = 'E:/graph data/type1平衡数据/'
    if isbalance:
        node_feature_label = pd.read_csv(pwd + 'node_feature_label__balance.csv', index_col = 0)
    else:
        node_feature_label = pd.read_csv(pwd + 'node_feature_label__nobalance.csv', index_col = 0)
    
    train_test_id_idx = np.load('E:/graph data/type1平衡数据/' + task + balance + '__testlabel0_knn_edge_train_test_index_all.npz', allow_pickle = True)
    train_index_all = train_test_id_idx['train_index_all']##k=3的所有5flod数据放在一起
    test_index_all = train_test_id_idx['test_index_all']
    

    
    num_node = node_feature_label.shape[0]   ###节点总数3508
    node_feat = node_feature_label.iloc[:, 3:]   ####所有的节点特征3508x848
    label = node_feature_label['label']   ##所有的节点标签  3508
    
    print('num_node',num_node)
    print('node_feat',node_feat)
    print(label)

    mirna_ids = list(set(node_feature_label['Drug']))  ##184   184种药物对应的序号
    disease_ids = list(set(node_feature_label['Mutation']))###661  661种突变对应的序号
    random.shuffle(mirna_ids) #将序列的所有元素随机排序。
    random.shuffle(disease_ids)
    print('# Drug = {} | Mutation = {}'.format(len(mirna_ids), len(disease_ids)))
    
    #print('mirna_ids',mirna_ids)
    #print('disease_ids',disease_ids)
    
    
    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: Drug = {} | Mutation = {}'.format(mirna_test_num, disease_test_num))
    ##从全部数据中选择1/5的测试集数据
    
    
    return node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all

In [3]:
def generate_graphsaint_data(task, train_index_all, test_index_all, node_feat, n_neigh, label, num_node, balance):

    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all): #train_index与test_index为下标
        # read knn_graph
        pwd = 'E:/graph data/type1平衡数据/'
        knn_graph_file = 'task_' + task + balance + '__testlabel0_knn' + str(n_neigh) + 'neighbors_edge__fold' + str(fold) + '.npz'
        knn_neighbors_graph = sp.load_npz(pwd + knn_graph_file)

        edge_src_dst = knn_neighbors_graph.nonzero()
        #print(edge_src_dst)
        
        #save dir
        save_dir = 'E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_' + task + balance + '__testlabel0_' + str(n_neigh) + 'knn_edge_fold' + str(fold) + '/'

        try:
            os.mkdir(save_dir)
        except OSError as error:
            print(error, save_dir)    
        ##先执行try的语句   如果错误  就执行except的语句   程序继续向下执行    
            
            
            
        # feats.npy，不需要自己标准化！因为在utils.py中的load_data中有标准化的步骤哦！
        feats = np.array(node_feat)
        np.save(save_dir + 'feats.npy', feats)
        #print('feats',feats)  ##feats   节点特征3508x848
        
        try:
            train_idx, test_idx = train_idx.tolist(), test_idx.tolist()
        except:
            train_idx, test_idx = train_idx, test_idx
            
        # role.json
        role = dict()
        role['tr'] = train_idx   ##2806
        role['va'] = test_idx    ###702
        role['te'] = test_idx    ###702 
        with open(save_dir + 'role.json','w') as f:
            json.dump(role, f)
        #print('json.dump(role, f)',role)   
        #print('role',len(train_idx))

        # class_map.json
        y = np.array(label)
        #print('y',len(y))
        class_map = dict()
        for i in range(num_node):
            class_map[str(i)] = y[i].tolist()
        with open(save_dir + 'class_map.json', 'w') as f:
            json.dump(class_map, f)
        #print('class_map:',class_map)
        
            
        # adj_*.npz
        train_idx_set = set(train_idx)
        test_idx_set = set(test_idx)
        
        row_full, col_full = edge_src_dst[0], edge_src_dst[1]
        
        row_train = []
        col_train = []
        row_val = []
        col_val = []
        for i in tqdm(range(row_full.shape[0])):
            if row_full[i] in train_idx_set and col_full[i] in train_idx_set:
                row_train.append(row_full[i])
                col_train.append(col_full[i])
            if row_full[i] in test_idx_set and col_full[i] in test_idx_set:
                row_val.append(row_full[i])
                col_val.append(col_full[i])

        row_train = np.array(row_train)
        col_train = np.array(col_train)
        row_val = np.array(row_val)
        col_val = np.array(col_val)
        dtype = np.bool

        adj_full = sp.coo_matrix(
            (
                np.ones(row_full.shape[0], dtype=dtype),
                (row_full, col_full),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        #print('adj_full',adj_full)
        

        adj_train = sp.coo_matrix(
            (
                np.ones(row_train.shape[0], dtype=dtype),
                (row_train, col_train),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        
        #print('adj_train',adj_train)

        adj_val = sp.coo_matrix(
            (
                np.ones(row_val.shape[0], dtype=dtype),
                (row_val, col_val),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        #print('adj_val',adj_val)

        #print('adj_full  num edges:', adj_full.nnz)
        #print('adj_val   num edges:', adj_val.nnz)
        #print('adj_train num edges:', adj_train.nnz)
        sp.save_npz(save_dir + 'adj_full.npz', adj_full)
        sp.save_npz(save_dir + 'adj_train.npz', adj_train)
        sp.save_npz(save_dir + 'adj_val.npz', adj_val) # adj_val not used in GraphSAINT source code

        fold += 1
    
    print('--Complete--', fold)
    return   feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst

In [4]:
def run(task, isbalance):
    
    if isbalance:
        balance = ''
    else:
        balance = '__nobalance'

    #for n_neigh in [1 ,3, 5, 7, 10, 15]:
    for n_neigh in [3, 5, 7, 10, 15]:

        node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all = obtain_data(task, 
                                                                                                                              isbalance,
                                                                                                                              balance)
        feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = generate_graphsaint_data(task, 
                                                                                        train_index_all, 
                                                                                        test_index_all, 
                                                                                        node_feat, 
                                                                                        n_neigh, 
                                                                                        label, 
                                                                                        num_node, 
                                                                                        balance)
    return node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst

# RUN balance

In [5]:
for task in ['Tp']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance =True)

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1     0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2     0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3     0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4     0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1666  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1667  0.929509  0.993179  0.886432  0.955361  0.972236  0.877694  0.959136   
1668  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   
1669  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   

           7_x       8_x       9_x  ...

100%|██████████| 5010/5010 [00:00<00:00, 1010699.97it/s]


[WinError 183] 当文件已存在时，无法创建该文件。: 'E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_3knn_edge_fold1/' E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_3knn_edge_fold1/


100%|██████████| 5010/5010 [00:00<00:00, 1014603.98it/s]


[WinError 183] 当文件已存在时，无法创建该文件。: 'E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_3knn_edge_fold2/' E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_3knn_edge_fold2/


...

[WinError 183] 当文件已存在时，无法创建该文件。: 'E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_15knn_edge_fold3/' E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_15knn_edge_fold3/


100%|██████████| 25050/25050 [00:00<00:00, 1003230.39it/s]


[WinError 183] 当文件已存在时，无法创建该文件。: 'E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_15knn_edge_fold4/' E:/graph data/type1平衡数据/type1平衡数据数据预处理模型/task_Tp__testlabel0_15knn_edge_fold4/


100%|██████████| 25050/25050 [00:00<00:00, 1047812.62it/s]

--Complete-- 5


In [16]:
for task in ['Td']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = True)

  0%|                                                                                         | 0/5010 [00:00<?, ?it/s]

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1     0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2     0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3     0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4     0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1666  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1667  0.929509  0.993179  0.886432  0.955361  0.972236  0.877694  0.959136   
1668  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   
1669  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   

           7_x       8_x       9_x  ...

100%|██████████████████████████████████████████████████████████████████████████| 5010/5010 [00:00<00:00, 717550.39it/s]


--Complete-- 5


  0%|                                                                                         | 0/8350 [00:00<?, ?it/s]

...

--Complete-- 5


  0%|                                                                                        | 0/25050 [00:00<?, ?it/s]

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1     0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2     0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3     0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4     0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1666  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1667  0.929509  0.993179  0.886432  0.955361  0.972236  0.877694  0.959136   
1668  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   
1669  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   

           7_x       8_x       9_x  ...

100%|████████████████████████████████████████████████████████████████████████| 25050/25050 [00:00<00:00, 866091.69it/s]

--Complete-- 5


In [17]:
for task in ['Tm']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = True)

100%|██████████████████████████████████████████████████████████████████████████| 5010/5010 [00:00<00:00, 837389.94it/s]

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1     0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2     0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3     0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4     0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1666  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1667  0.929509  0.993179  0.886432  0.955361  0.972236  0.877694  0.959136   
1668  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   
1669  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   

           7_x       8_x       9_x  ...


100%|██████████████████████████████████████████████████████████████████████████| 5010/5010 [00:00<00:00, 837223.12it/s]


--Complete-- 5


  0%|                                                                                         | 0/8350 [00:00<?, ?it/s]

...

--Complete-- 5


  0%|                                                                                        | 0/25050 [00:00<?, ?it/s]

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1     0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2     0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3     0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4     0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1666  0.859920  0.909371  0.997017  0.973323  0.969097  0.997544  0.972053   
1667  0.929509  0.993179  0.886432  0.955361  0.972236  0.877694  0.959136   
1668  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   
1669  0.960399  0.973907  0.974810  0.000000  0.999249  0.973968  0.987774   

           7_x       8_x       9_x  ...

100%|████████████████████████████████████████████████████████████████████████| 25050/25050 [00:00<00:00, 866091.69it/s]

--Complete-- 5


# RUN nobalance

In [22]:
for task in ['Tp']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = False)
    


num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

            7_x       8_x 

100%|██████████| 31572/31572 [00:00<00:00, 791399.87it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 52620/52620 [00:00<00:00, 732786.42it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

...

--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 105240/105240 [00:00<00:00, 620716.04it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 157860/157860 [00:00<00:00, 608778.93it/s]


--Complete-- 5


In [219]:
node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = 'Tm' , isbalance = False)

# Drug = 184 | Mutation = 661
# Test: Drug = 36 | Mutation = 132


100%|██████████| 121624/121624 [00:00<00:00, 743593.26it/s]


adj_full  num edges: 121624
adj_val   num edges: 23796
adj_train num edges: 97828


100%|██████████| 121624/121624 [00:00<00:00, 771833.67it/s]


adj_full  num edges: 121624
adj_val   num edges: 23796
adj_train num edges: 97828


...

adj_full  num edges: 1824360
adj_val   num edges: 239547
adj_train num edges: 1349590


100%|██████████| 1824360/1824360 [00:02<00:00, 817361.01it/s]


adj_full  num edges: 1824360
adj_val   num edges: 240510
adj_train num edges: 1354072


100%|██████████| 1824360/1824360 [00:02<00:00, 819623.84it/s]


adj_full  num edges: 1824360
adj_val   num edges: 272914
adj_train num edges: 1304354
--Complete-- 5


In [226]:
# for n_neigh in [1, 3, 5, 7, 15]: 
node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = 'Td', isbalance = False)

# Drug = 184 | Mutation = 661
# Test: Drug = 36 | Mutation = 132


100%|██████████| 121624/121624 [00:00<00:00, 734465.32it/s]


adj_full  num edges: 121624
adj_val   num edges: 24288
adj_train num edges: 97336


100%|██████████| 121624/121624 [00:00<00:00, 743468.63it/s]


adj_full  num edges: 121624
adj_val   num edges: 24288
adj_train num edges: 97336


...

adj_full  num edges: 1824360
adj_val   num edges: 91909
adj_train num edges: 1187590


100%|██████████| 1824360/1824360 [00:02<00:00, 895802.15it/s]


adj_full  num edges: 1824360
adj_val   num edges: 91562
adj_train num edges: 1187288


100%|██████████| 1824360/1824360 [00:02<00:00, 878344.01it/s]


adj_full  num edges: 1824360
adj_val   num edges: 93017
adj_train num edges: 1183202
--Complete-- 5
